In [1]:

from langchain.utilities.duckduckgo_search import DuckDuckGoSearchAPIWrapper
import yfinance
import json


def get_ticker(inputs):
    ddg = DuckDuckGoSearchAPIWrapper()
    company_name = inputs["company_name"]
    return ddg.run(f"Ticker symbol of {company_name}")


def get_income_statement(inputs):
    ticker = inputs["ticker"]
    stock = yfinance.Ticker(ticker)
    return json.dumps(stock.income_stmt.to_json())


def get_balance_sheet(inputs):
    ticker = inputs["ticker"]
    stock = yfinance.Ticker(ticker)
    return json.dumps(stock.balance_sheet.to_json())


def get_daily_stock_performance(inputs):
    ticker = inputs["ticker"]
    stock = yfinance.Ticker(ticker)
    return json.dumps(stock.history(period="3mo").to_json())


functions_map = {
    "get_ticker": get_ticker,
    "get_income_statement": get_income_statement,
    "get_balance_sheet": get_balance_sheet,
    "get_daily_stock_performance": get_daily_stock_performance,
}

functions = [
    {
        "type": "function",
        "function": {
            "name": "get_ticker",
            "description": "Given the name of a company returns its ticker symbol",
            "parameters": {
                "type": "object",
                "properties": {
                    "company_name": {
                        "type": "string",
                        "description": "The name of the company",
                    }
                },
                "required": ["company_name"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_income_statement",
            "description": "Given a ticker symbol (i.e AAPL) returns the company's income statement.",
            "parameters": {
                "type": "object",
                "properties": {
                    "ticker": {
                        "type": "string",
                        "description": "Ticker symbol of the company",
                    },
                },
                "required": ["ticker"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_balance_sheet",
            "description": "Given a ticker symbol (i.e AAPL) returns the company's balance sheet.",
            "parameters": {
                "type": "object",
                "properties": {
                    "ticker": {
                        "type": "string",
                        "description": "Ticker symbol of the company",
                    },
                },
                "required": ["ticker"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_daily_stock_performance",
            "description": "Given a ticker symbol (i.e AAPL) returns the performance of the stock for the last 100 days.",
            "parameters": {
                "type": "object",
                "properties": {
                    "ticker": {
                        "type": "string",
                        "description": "Ticker symbol of the company",
                    },
                },
                "required": ["ticker"],
            },
        },
    },
]

In [2]:
from openai import OpenAI
client = OpenAI()

# assistant = client.beta.assistants.create(
#     name="Investor Assistant",
#     instructions="You help users do research on publicly traded companies and you help users decide if they should buy the stock or not.",
#     model="gpt-4-1106-preview",
#     tools=functions,
# )
# --------------------------------------
# assistant = client.beta.assistants.create(
#     name="Book Assistant",
#     instructions="You help users with their question on the files they upload.",
#     model="gpt-4-1106-preview",
#     tools=[{"type": "retrieval"}],
# )
# assistant_id = "asst_G6huadGfMYhX4mckzAsjKfij"

In [8]:
thread = client.beta.threads.create(
    messages=[
        {
            "role": "user",
            # "content": "I want to know if the Salesforce stock is a good buy",
            "content": "I want you to help me with this file",
        }
    ]
)
thread

Thread(id='thread_0aq9dSS2Zlaa1E2BDwy8rOJh', created_at=1726204093, metadata={}, object='thread', tool_resources=ToolResources(code_interpreter=None, file_search=None))

In [ ]:
file = client.files.create(
    file=client.file_from_path("./files/chapter_one.txt"), purpose="assistants"
)
file

In [ ]:
client.beta.threads.messages.create(
    thread_id=thread.id, role="user", content="", file_ids=[file.id]
)

In [9]:
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant_id,
)
run

Run(id='run_xn1aPih5yZ8KywotP90NMIln', assistant_id='asst_G6huadGfMYhX4mckzAsjKfij', cancelled_at=None, completed_at=None, created_at=1726204095, expires_at=1726204695, failed_at=None, incomplete_details=None, instructions='You help users do research on publicly traded companies and you help users decide if they should buy the stock or not.', last_error=None, max_completion_tokens=None, max_prompt_tokens=None, metadata={}, model='gpt-4-1106-preview', object='thread.run', parallel_tool_calls=True, required_action=None, response_format='auto', started_at=None, status='queued', thread_id='thread_0aq9dSS2Zlaa1E2BDwy8rOJh', tool_choice='auto', tools=[FunctionTool(function=FunctionDefinition(name='get_ticker', description='Given the name of a company returns its ticker symbol', parameters={'type': 'object', 'properties': {'company_name': {'type': 'string', 'description': 'The name of the company'}}, 'required': ['company_name']}, strict=False), type='function'), FunctionTool(function=Functio

In [5]:
def get_run(run_id, thread_id):
    return client.beta.threads.runs.retrieve(
        run_id=run_id,
        thread_id=thread_id,
    )


def send_message(thread_id, content):
    return client.beta.threads.messages.create(
        thread_id=thread_id, role="user", content=content
    )


def get_messages(thread_id):
    messages = client.beta.threads.messages.list(thread_id=thread_id).data
    messages.reverse()
    for message in messages:
        print(f"{message.role}: ", end=' ')
    if message.content:
        print(f"{message.content[0].text.value}")

def get_tool_outputs(run_id, thread_id):
    run = get_run(run_id, thread_id)
    outputs = []
    for action in run.required_action.submit_tool_outputs.tool_calls:
        action_id = action.id
        function = action.function
        # print(action)
        print(f"Calling function: {function.name} with arg {function.arguments}")
        outputs.append(
            {
                "output": functions_map[function.name](json.loads(function.arguments)),
                "tool_call_id": action_id,
            }
        )
    return outputs

def submit_tool_outputs(run_id, thread_id):
    outpus = get_tool_outputs(run_id, thread_id)
    return client.beta.threads.runs.submit_tool_outputs(
        run_id=run_id, thread_id=thread_id, tool_outputs=outpus
    )




In [196]:
send_message(thread.id, "Go ahead")

Message(id='msg_m8Z8lFKgAUaL2fdX5jHHDYXG', assistant_id=None, attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='Go ahead'), type='text')], created_at=1726203884, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_LDlTnBmmrrP2OWZ78q4xySNq')

In [28]:
get_run(run.id, thread.id).status

'completed'

In [35]:
get_messages(thread.id)

user:  assistant:  user:  Now I want to know if Cloudflare is a good buy.


In [30]:
get_tool_outputs(run.id, thread.id)

AttributeError: 'NoneType' object has no attribute 'submit_tool_outputs'

In [33]:
submit_tool_outputs(run.id, thread.id)

AttributeError: 'NoneType' object has no attribute 'submit_tool_outputs'

In [27]:
send_message(thread.id, "Now I want to know if Cloudflare is a good buy.")

Message(id='msg_btzMTA4SI9XdIxE6OzHrLvwE', assistant_id=None, attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='Now I want to know if Cloudflare is a good buy.'), type='text')], created_at=1726204220, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_0aq9dSS2Zlaa1E2BDwy8rOJh')